## Segmenting and Clustering Neighborhoods in Toronto

### Part 01 of the Assignment

In [1]:
#importing required libraries

import pandas as pd
import requests

In [2]:
#acquiring the data, and converting it into a data frame. 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
temp = requests.get(url)

df_temp = pd.read_html(temp.content, header=0)[0]

#Ignore Boroughs where value is 'Not assigned'.
df = df_temp[df_temp.Borough != 'Not assigned']

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df.loc[df['Neighborhood'] == "Not assigned"]

,Postal Code,Borough,Neighborhood


#### No Row found where Neighborhood is Not Assigned.

In [4]:
#more than one neighborhood under the same postal code
df_tor = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x:', '.join(x))
df_tor = df_tor.reset_index()
df_tor


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [5]:
#printing size of dataframe
print ("The Dataframe df_tor has", df_tor.shape[0], 'rows.')

The Dataframe df_tor has 103 rows.




### Part 02 of the Assignment

Collecting Lat-Long of the Locations from csv file containing Geospatial Data

In [6]:
df_latlong = pd.read_csv('https://cocl.us/Geospatial_data')
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge df_latlong with original dataframe df_tor.

In [7]:
df_tor = pd.merge(df_tor, df_latlong, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df_tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 03 of the Assignment 

Opting to work only with boroughs that contain the name 'Toronto'